We made our predictions in phase2_submissions1.csv. Now let's back out what those submissions are in the bracket.

In [405]:
import pandas as pd

In [565]:
df_pred = pd.read_csv('../../data/predictions/phase2_submissions1.csv')
df_pred.head()

,ID,Pred
0,2021_1101_1104,0.003117
1,2021_1101_1111,0.805002
2,2021_1101_1116,0.000475
3,2021_1101_1124,0.008319
4,2021_1101_1140,0.012145


Excellent, now we have our submission file into a dataframe with ID and Predictions. From here we'll need to bring in the first round teams and see who we predict will win.

In [566]:
#read in the team data
df_teams = pd.read_csv('../../data/raw/MDataFiles_Stage2/MTeams.csv')
df_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021
1,1102,Air Force,1985,2021
2,1103,Akron,1985,2021
3,1104,Alabama,1985,2021
4,1105,Alabama A&M,2000,2021


In [567]:
#read in the tournament slot data
df_slots = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneySlots.csv')
df_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [568]:
df_slots2021 = df_slots[df_slots['Season']==2021]
df_slots2021 = df_slots2021.reset_index(drop=True)
df_slots2021.head(48)

,Season,Slot,StrongSeed,WeakSeed
0,2021,R1W1,W01,W16
1,2021,R1W2,W02,W15
2,2021,R1W3,W03,W14
3,2021,R1W4,W04,W13
4,2021,R1W5,W05,W12
5,2021,R1W6,W06,W11
6,2021,R1W7,W07,W10
7,2021,R1W8,W08,W09
8,2021,R1X1,X01,X16
9,2021,R1X2,X02,X15


In [569]:
df_slots2021.tail(10)

,Season,Slot,StrongSeed,WeakSeed
57,2021,R4X1,R3X1,R3X2
58,2021,R4Y1,R3Y1,R3Y2
59,2021,R4Z1,R3Z1,R3Z2
60,2021,R5WX,R4W1,R4X1
61,2021,R5YZ,R4Y1,R4Z1
62,2021,R6CH,R5WX,R5YZ
63,2021,W11,W11a,W11b
64,2021,W16,W16a,W16b
65,2021,X11,X11a,X11b
66,2021,X16,X16a,X16b


In [570]:
#read in the tournament seed data data
df_seeds = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneySeeds.csv')
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [571]:
df_seeds2021 = df_seeds[df_seeds['Season']==2021]
df_seeds2021 = df_seeds2021.reset_index(drop=True)
df_seeds2021.head()

,Season,Seed,TeamID
0,2021,W01,1276
1,2021,W02,1104
2,2021,W03,1400
3,2021,W04,1199
4,2021,W05,1160


Rather than messing with the play-in games, let's take 11a and 16a as seed 11 and 16. Then we don't need to worry about it. 

In [572]:
df_seeds2021.Seed.replace(['W11a','W16a','X11a','X16a'],['W11','W16','X11','X16'],inplace=True)

I think with these three dataframes, we can write a function that will take the slot information to find out which slots play eachother in each round. Then we use the seed data frame to look up which TeamID is associated with the slots. Then we look at our predictions to find out who we predicted to win. From there we should be able to label the winning teams with their seed for the next round and continue on. 

In [573]:
'''Given two integers 0,8 for W round 1, return a list of Strong Seeds and a list of Weak Seeds'''
def match_upID(int1:int,int2:int):
    StrongSeed = df_slots2021.StrongSeed[int1:int2]
    WeakSeed = df_slots2021.WeakSeed[int1:int2]
    TeamID_Strong = [int(df_seeds2021.TeamID[df_seeds2021.Seed==seed]) for seed in StrongSeed]
    TeamID_Weak = [int(df_seeds2021.TeamID[df_seeds2021.Seed==seed]) for seed in WeakSeed]
    return TeamID_Strong,TeamID_Weak

In [574]:
'''Given the TeamID of the strong and weak seed in the match up, which TeamName 
was predicted to be the winner''' 
def winners(strong:int,weak:int):
    if strong > weak:
        id_pred = '2021_'+str(weak)+'_'+str(strong)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        print(pred)
        if pred < 0.5:
            win_teamID = strong
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print(win_team)
        elif pred > 0.5:
            win_teamID = weak
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print(win_team)
    elif strong < weak:
        id_pred = '2021_'+str(strong)+'_'+str(weak)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        print(pred)
        if pred < 0.5:
            win_teamID = weak
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print(win_team)
        elif pred > 0.5:
            win_teamID = strong
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print(win_team)
    return win_team

In [575]:
'''Take the list of Strong TeamIDs and Weak TeamIDs, and use the winners function 
to make a dictionary of the winning team and the rank for the next round.'''
def round_winners(strong_teams:list, weak_teams:list, end_rank:str):
    rank = [end_rank+'{}'.format(i+1) for i in range(8)]
    winning_teams = []
    for i in range(len(strong_teams)):
        strong_id = strong_teams[i]
        weak_id = weak_teams[i]
        winner = winners(strong_id, weak_id)
        winning_teams.append(str(winner))
    round_wins = dict(zip(winning_teams,rank))
    return round_wins

# Region: W
# Round: 1

In [576]:
TeamID_Strong, TeamID_Weak = match_upID(0,8)
TeamID_Strong, TeamID_Weak

([1276, 1104, 1400, 1199, 1160, 1140, 1163, 1261],
 [1291, 1233, 1101, 1422, 1207, 1277, 1268, 1382])

In [577]:
round1_w = round_winners(TeamID_Strong,TeamID_Weak,'R1W')
round1_w

0.9950100000000001
Michigan
0.89418155
Alabama
0.123211145
Texas
0.79954004
Florida St
0.49994612
Georgetown
0.19828413
Michigan St
0.8232264000000001
Connecticut
0.523091
LSU


{'Michigan': 'R1W1',
 'Alabama': 'R1W2',
 'Texas': 'R1W3',
 'Florida St': 'R1W4',
 'Georgetown': 'R1W5',
 'Michigan St': 'R1W6',
 'Connecticut': 'R1W7',
 'LSU': 'R1W8'}

# Region: W
# Round: 2

In [578]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round1_w]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(4)]
TeamID_Weak = [IDs[i] for i in reversed(range(4,8))]
TeamID_Strong, TeamID_Weak

[1276, 1104, 1400, 1199, 1207, 1277, 1163, 1261]


([1276, 1104, 1400, 1199], [1261, 1163, 1277, 1207])

In [579]:
round2_w = round_winners(TeamID_Strong,TeamID_Weak,'R2W')
round2_w

0.022041498
Michigan
0.034147073
Connecticut
0.099247396
Texas
0.69937104
Florida St


{'Michigan': 'R2W1',
 'Connecticut': 'R2W2',
 'Texas': 'R2W3',
 'Florida St': 'R2W4'}

# Region: W
# Round: 3

In [580]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round2_w]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(2)]
TeamID_Weak = [IDs[i] for i in reversed(range(2,4))]
TeamID_Strong, TeamID_Weak

[1276, 1163, 1400, 1199]


([1276, 1163], [1199, 1400])

In [581]:
round3_w = round_winners(TeamID_Strong,TeamID_Weak,'R3W')
round3_w

0.010987321000000001
Michigan
0.88587695
Connecticut


{'Michigan': 'R3W1', 'Connecticut': 'R3W2'}

# Region: W
# Round: 4

In [582]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round3_w]
print(IDs)
TeamID_Strong = IDs[0]
TeamID_Weak = IDs[1]
TeamID_Strong, TeamID_Weak

[1276, 1163]


(1276, 1163)

In [583]:
rank_for_5_w = 'R4W1'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_w4 = winner
round4_w = {winner_w4,rank_for_5_w}
round4_w

0.03231762
Michigan


{'Michigan', 'R4W1'}

# Region: X
# Round: 1

In [584]:
TeamID_Strong, TeamID_Weak = match_upID(8,16)
TeamID_Strong, TeamID_Weak

([1211, 1234, 1242, 1438, 1166, 1425, 1332, 1328],
 [1111, 1213, 1186, 1325, 1364, 1179, 1433, 1281])

In [585]:
round1_x = round_winners(TeamID_Strong,TeamID_Weak,'R1X')
round1_x

0.0006938986500000001
Gonzaga
0.006273976999999999
Iowa
7.169798e-05
Kansas
0.1676381
Virginia
0.9472164000000001
Creighton
0.640927
Drake
0.44566300000000003
VCU
0.5576659
Missouri


{'Gonzaga': 'R1X1',
 'Iowa': 'R1X2',
 'Kansas': 'R1X3',
 'Virginia': 'R1X4',
 'Creighton': 'R1X5',
 'Drake': 'R1X6',
 'VCU': 'R1X7',
 'Missouri': 'R1X8'}

# Region: X
# Round: 2

In [586]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round1_x]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(4)]
TeamID_Weak = [IDs[i] for i in reversed(range(4,8))]
TeamID_Strong, TeamID_Weak

[1211, 1234, 1242, 1438, 1166, 1179, 1433, 1281]


([1211, 1234, 1242, 1438], [1281, 1433, 1179, 1166])

In [587]:
round2_x = round_winners(TeamID_Strong,TeamID_Weak,'R2X')
round2_x

0.7196403
Gonzaga
0.97314507
Iowa
0.0040316815
Kansas
0.8855226
Creighton


{'Gonzaga': 'R2X1', 'Iowa': 'R2X2', 'Kansas': 'R2X3', 'Creighton': 'R2X4'}

# Region: X
# Round: 3

In [588]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round2_x]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(2)]
TeamID_Weak = [IDs[i] for i in reversed(range(2,4))]
TeamID_Strong, TeamID_Weak

[1211, 1234, 1242, 1166]


([1211, 1234], [1166, 1242])

In [589]:
round3_x = round_winners(TeamID_Strong,TeamID_Weak,'R3X')
round3_x

0.00019070278000000003
Gonzaga
0.27701974
Kansas


{'Gonzaga': 'R3X1', 'Kansas': 'R3X2'}

# Region: X
# Round: 4

In [590]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round3_x]
print(IDs)
TeamID_Strong = IDs[0]
TeamID_Weak = IDs[1]
TeamID_Strong, TeamID_Weak

[1211, 1242]


(1211, 1242)

In [591]:
rank_for_5_x = 'R4X1'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_x4 = winner
round4_x = {winner_x4,rank_for_5_x}
round4_x

0.24811538
Kansas


{'Kansas', 'R4X1'}

# Region: Y
# Round: 1

In [592]:
TeamID_Strong, TeamID_Weak = match_upID(16,24)
TeamID_Strong, TeamID_Weak

([1228, 1222, 1452, 1329, 1397, 1361, 1155, 1260],
 [1180, 1156, 1287, 1251, 1333, 1393, 1353, 1210])

In [593]:
round1_y = round_winners(TeamID_Strong,TeamID_Weak,'R1Y')
round1_y

0.0015877353
Illinois
0.0011223365
Houston
0.008144338000000001
West Virginia
0.032172132
Oklahoma St
0.020022621
Tennessee
0.2729452
Syracuse
0.07893299
Rutgers
0.10876051
Loyola-Chicago


{'Illinois': 'R1Y1',
 'Houston': 'R1Y2',
 'West Virginia': 'R1Y3',
 'Oklahoma St': 'R1Y4',
 'Tennessee': 'R1Y5',
 'Syracuse': 'R1Y6',
 'Rutgers': 'R1Y7',
 'Loyola-Chicago': 'R1Y8'}

# Region: Y
# Round: 2

In [594]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round1_y]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(4)]
TeamID_Weak = [IDs[i] for i in reversed(range(4,8))]
TeamID_Strong, TeamID_Weak

[1228, 1222, 1452, 1329, 1397, 1393, 1353, 1260]


([1228, 1222, 1452, 1329], [1260, 1353, 1393, 1397])

In [595]:
round2_y = round_winners(TeamID_Strong,TeamID_Weak,'R2Y')
round2_y

0.7780925
Illinois
0.99727494
Houston
0.6835757
Syracuse
0.05393996
Tennessee


{'Illinois': 'R2Y1',
 'Houston': 'R2Y2',
 'Syracuse': 'R2Y3',
 'Tennessee': 'R2Y4'}

# Region: Y
# Round: 3

In [596]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round2_y]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(2)]
TeamID_Weak = [IDs[i] for i in reversed(range(2,4))]
TeamID_Strong, TeamID_Weak

[1228, 1222, 1393, 1397]


([1228, 1222], [1397, 1393])

In [597]:
round3_y = round_winners(TeamID_Strong,TeamID_Weak,'R3Y')
round3_y

0.9706971
Illinois
0.9158546000000001
Houston


{'Illinois': 'R3Y1', 'Houston': 'R3Y2'}

# Region: Y
# Round: 4

In [598]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round3_y]
print(IDs)
TeamID_Strong = IDs[0]
TeamID_Weak = IDs[1]
TeamID_Strong, TeamID_Weak

[1228, 1222]


(1228, 1222)

In [599]:
rank_for_5_y = 'R4Y1'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_y4 = winner
round4_y = {winner_y4,rank_for_5_y}
round4_y

0.0031249332
Illinois


{'Illinois', 'R4Y1'}

# Region: Z
# Round: 1

In [600]:
TeamID_Strong, TeamID_Weak = match_upID(24,32)
TeamID_Strong, TeamID_Weak

([1124, 1326, 1116, 1345, 1437, 1403, 1196, 1314],
 [1216, 1331, 1159, 1317, 1457, 1429, 1439, 1458])

In [601]:
round1_z = round_winners(TeamID_Strong,TeamID_Weak,'R1Z')
round1_z

0.9941087
Baylor
0.96110046
Ohio St
0.071070045
Colgate
0.011253725
Purdue
0.93505806
Villanova
0.84096706
Texas Tech
0.40585133
Virginia Tech
0.19406347
Wisconsin


{'Baylor': 'R1Z1',
 'Ohio St': 'R1Z2',
 'Colgate': 'R1Z3',
 'Purdue': 'R1Z4',
 'Villanova': 'R1Z5',
 'Texas Tech': 'R1Z6',
 'Virginia Tech': 'R1Z7',
 'Wisconsin': 'R1Z8'}

# Region: Z
# Round: 2

In [602]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round1_z]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(4)]
TeamID_Weak = [IDs[i] for i in reversed(range(4,8))]
TeamID_Strong, TeamID_Weak

[1124, 1326, 1159, 1345, 1437, 1403, 1439, 1458]


([1124, 1326, 1159, 1345], [1458, 1439, 1403, 1437])

In [603]:
round2_z = round_winners(TeamID_Strong,TeamID_Weak,'R2Z')
round2_z

0.91007924
Baylor
0.95308226
Ohio St
0.018551443
Texas Tech
0.04858126
Villanova


{'Baylor': 'R2Z1',
 'Ohio St': 'R2Z2',
 'Texas Tech': 'R2Z3',
 'Villanova': 'R2Z4'}

# Region: Z
# Round: 3

In [604]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round2_z]
print(IDs)
TeamID_Strong = [IDs[i] for i in range(2)]
TeamID_Weak = [IDs[i] for i in reversed(range(2,4))]
TeamID_Strong, TeamID_Weak

[1124, 1326, 1403, 1437]


([1124, 1326], [1437, 1403])

In [605]:
round3_z = round_winners(TeamID_Strong,TeamID_Weak,'R3Z')
round3_z

0.7910138000000001
Baylor
0.8542736999999999
Ohio St


{'Baylor': 'R3Z1', 'Ohio St': 'R3Z2'}

# Region: Z
# Round: 4

In [606]:
IDs = [df_teams.loc[df_teams.TeamName==key,'TeamID'].iloc[0] for key in round3_z]
print(IDs)
TeamID_Strong = IDs[0]
TeamID_Weak = IDs[1]
TeamID_Strong, TeamID_Weak

[1124, 1326]


(1124, 1326)

In [607]:
rank_for_5_z = 'R4Z1'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_z4 = winner
round4_z = {winner_z4,rank_for_5_z}
round4_z

0.5710550999999999
Baylor


{'Baylor', 'R4Z1'}

# Region: W-X

In [608]:
TeamID_Strong = df_teams.loc[df_teams.TeamName==winner_w4,'TeamID'].iloc[0]
TeamID_Weak = df_teams.loc[df_teams.TeamName==winner_x4,'TeamID'].iloc[0]
TeamID_Strong, TeamID_Weak

(1276, 1242)

In [609]:
rank_for_ch_wx = 'R5WX'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_wx = winner
round5_wx = {winner_wx,rank_for_ch_wx}
round5_wx

0.0031281090000000004
Michigan


{'Michigan', 'R5WX'}

# Region: Y-Z

In [610]:
TeamID_Strong = df_teams.loc[df_teams.TeamName==winner_y4,'TeamID'].iloc[0]
TeamID_Weak = df_teams.loc[df_teams.TeamName==winner_z4,'TeamID'].iloc[0]
TeamID_Strong, TeamID_Weak

(1228, 1124)

In [611]:
rank_for_ch_yz = 'R5YZ'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_yz = winner
round5_yz = {winner_yz,rank_for_ch_yz}
round5_yz

0.020854726
Illinois


{'Illinois', 'R5YZ'}

# Championship

In [612]:
TeamID_Strong = df_teams.loc[df_teams.TeamName==winner_wx,'TeamID'].iloc[0]
TeamID_Weak = df_teams.loc[df_teams.TeamName==winner_yz,'TeamID'].iloc[0]
TeamID_Strong, TeamID_Weak

(1276, 1228)

In [613]:
rank_for_ch_yz = 'R6CH'
strong_id = TeamID_Strong
weak_id = TeamID_Weak
winner = winners(strong_id, weak_id)
winner_ch = winner
round6_ch = {winner_ch,rank_for_ch_yz}
round6_ch

0.015207883
Michigan


{'Michigan', 'R6CH'}

In [614]:
df_teams.loc[df_teams.TeamName=='UCLA','TeamID'].iloc[0]

1417

In [615]:
df_teams.loc[df_teams.TeamName=='BYU','TeamID'].iloc[0]

1140

In [616]:
ID_tag = '2021_1140_1417'

In [617]:
df_pred[df_pred.ID==ID_tag]

,ID,Pred
375,2021_1140_1417,0.545678


In [618]:
df_teams.loc[df_teams.TeamName=='Michigan St','TeamID'].iloc[0]

1277

In [619]:
ID_tag2 = '2021_1140_1277'

In [620]:
df_pred[df_pred.ID==ID_tag2]

,ID,Pred
351,2021_1140_1277,0.198284
